In [2]:
!pip install transformers datasets torch evaluate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertModel, AutoModelForCausalLM
from google.colab import drive
from datasets import Dataset, DatasetDict
import numpy
import torch
from huggingface_hub import login

In [4]:
power_data = pd.read_csv("/content/power-lv-train.tsv", sep="\t")

print("Power Data Distribution:")
print(power_data['label'].value_counts())

Power Data Distribution:
label
0    944
1    466
Name: count, dtype: int64


In [5]:
power_train, power_validate = train_test_split(power_data, train_size=0.9, test_size=0.1, stratify=power_data['label'])

print("Power Train Data Distribution:")
print(power_train['label'].value_counts())
print("Power Validate Data Distribution:")
print(power_validate['label'].value_counts())

Power Train Data Distribution:
label
0    850
1    419
Name: count, dtype: int64
Power Validate Data Distribution:
label
0    94
1    47
Name: count, dtype: int64


In [6]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize_lv(data):
  return tokenizer(data['text'], padding=True, truncation=True, max_length=512)

def tokenize_en(data):
  return tokenizer(data['text_en'], padding=True, truncation=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
dataset_training = Dataset.from_pandas(power_train)
dataset_validation = Dataset.from_pandas(power_validate)

dataset_training_lv = dataset_training.map(tokenize_lv, batched=True)
dataset_validation_lv = dataset_validation.map(tokenize_lv, batched=True)

dataset_training_en = dataset_training.map(tokenize_en, batched=True)
dataset_validation_en = dataset_validation.map(tokenize_en, batched=True)

Map:   0%|          | 0/1269 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

Map:   0%|          | 0/1269 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

In [8]:
training_arguments_lv = TrainingArguments(
    output_dir="./results_lv",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_lv",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    save_total_limit=1,
)

training_arguments_en = TrainingArguments(
    output_dir="./results_en",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_en",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=50,
    save_total_limit=1,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return classification_report(labels, preds, output_dict=True)

trainer_lv = Trainer(
    model=model,
    args=training_arguments_lv,
    train_dataset=dataset_training_lv,
    eval_dataset=dataset_validation_lv,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

trainer_en = Trainer(
    model=model,
    args=training_arguments_en,
    train_dataset=dataset_training_en,
    eval_dataset=dataset_validation_en,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-130f15aef0a8>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lv = Trainer(
<ipython-input-8-130f15aef0a8>:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_en = Trainer(


In [12]:
trainer_lv.train()
predictions_lv = trainer_lv.predict(dataset_validation_lv)
result_lv = numpy.argmax(predictions_lv.predictions, axis=-1)
labels_lv = dataset_validation_lv['label']
print(classification_report(labels_lv, result_lv))

trainer_en.train()
predictions_en = trainer_en.predict(dataset_validation_en)
result_en = numpy.argmax(predictions_en.predictions, axis=-1)
labels_en = dataset_validation_en['label']
print(classification_report(labels_en, result_en))

Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.573800,0.663515,"{'precision': 0.6727272727272727, 'recall': 0.7872340425531915, 'f1-score': 0.7254901960784313, 'support': 94.0}","{'precision': 0.3548387096774194, 'recall': 0.23404255319148937, 'f1-score': 0.28205128205128205, 'support': 47.0}",0.602837,"{'precision': 0.513782991202346, 'recall': 0.5106382978723405, 'f1-score': 0.5037707390648567, 'support': 141.0}","{'precision': 0.5667644183773215, 'recall': 0.6028368794326241, 'f1-score': 0.5776772247360481, 'support': 141.0}"
2,0.569700,0.656628,"{'precision': 0.7228915662650602, 'recall': 0.6382978723404256, 'f1-score': 0.6779661016949152, 'support': 94.0}","{'precision': 0.41379310344827586, 'recall': 0.5106382978723404, 'f1-score': 0.45714285714285713, 'support': 47.0}",0.595745,"{'precision': 0.5683423348566681, 'recall': 0.574468085106383, 'f1-score': 0.5675544794188861, 'support': 141.0}","{'precision': 0.6198587453261322, 'recall': 0.5957446808510638, 'f1-score': 0.6043583535108958, 'support': 141.0}"
3,0.553100,0.644122,"{'precision': 0.6875, 'recall': 0.9361702127659575, 'f1-score': 0.7927927927927928, 'support': 94.0}","{'precision': 0.5384615384615384, 'recall': 0.14893617021276595, 'f1-score': 0.23333333333333334, 'support': 47.0}",0.673759,"{'precision': 0.6129807692307692, 'recall': 0.5425531914893618, 'f1-score': 0.513063063063063, 'support': 141.0}","{'precision': 0.6378205128205128, 'recall': 0.6737588652482269, 'f1-score': 0.6063063063063063, 'support': 141.0}"


Trainer is attempting to log a value of "{'precision': 0.6727272727272727, 'recall': 0.7872340425531915, 'f1-score': 0.7254901960784313, 'support': 94.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.3548387096774194, 'recall': 0.23404255319148937, 'f1-score': 0.28205128205128205, 'support': 47.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.513782991202346, 'recall': 0.5106382978723405, 'f1-score': 0.5037707390648567, 'support': 141.0}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5667644183773215, 're

              precision    recall  f1-score   support

           0       0.69      0.94      0.79        94
           1       0.54      0.15      0.23        47

    accuracy                           0.67       141
   macro avg       0.61      0.54      0.51       141
weighted avg       0.64      0.67      0.61       141



Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.602900,0.672190,"{'precision': 0.6764705882352942, 'recall': 0.7340425531914894, 'f1-score': 0.7040816326530612, 'support': 94.0}","{'precision': 0.358974358974359, 'recall': 0.2978723404255319, 'f1-score': 0.32558139534883723, 'support': 47.0}",0.588652,"{'precision': 0.5177224736048266, 'recall': 0.5159574468085106, 'f1-score': 0.5148315140009492, 'support': 141.0}","{'precision': 0.5706385118149825, 'recall': 0.5886524822695035, 'f1-score': 0.5779148868849866, 'support': 141.0}"
2,0.559500,0.633802,"{'precision': 0.6936936936936937, 'recall': 0.8191489361702128, 'f1-score': 0.751219512195122, 'support': 94.0}","{'precision': 0.43333333333333335, 'recall': 0.2765957446808511, 'f1-score': 0.33766233766233766, 'support': 47.0}",0.638298,"{'precision': 0.5635135135135135, 'recall': 0.5478723404255319, 'f1-score': 0.5444409249287299, 'support': 141.0}","{'precision': 0.606906906906907, 'recall': 0.6382978723404256, 'f1-score': 0.6133671206841939, 'support': 141.0}"
3,0.472200,0.813168,"{'precision': 0.7469879518072289, 'recall': 0.6595744680851063, 'f1-score': 0.7005649717514124, 'support': 94.0}","{'precision': 0.4482758620689655, 'recall': 0.5531914893617021, 'f1-score': 0.49523809523809526, 'support': 47.0}",0.624113,"{'precision': 0.5976319069380972, 'recall': 0.6063829787234043, 'f1-score': 0.5979015334947538, 'support': 141.0}","{'precision': 0.6474172552278078, 'recall': 0.624113475177305, 'f1-score': 0.6321226795803067, 'support': 141.0}"


Trainer is attempting to log a value of "{'precision': 0.6764705882352942, 'recall': 0.7340425531914894, 'f1-score': 0.7040816326530612, 'support': 94.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.358974358974359, 'recall': 0.2978723404255319, 'f1-score': 0.32558139534883723, 'support': 47.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5177224736048266, 'recall': 0.5159574468085106, 'f1-score': 0.5148315140009492, 'support': 141.0}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5706385118149825, 'rec

              precision    recall  f1-score   support

           0       0.69      0.82      0.75        94
           1       0.43      0.28      0.34        47

    accuracy                           0.64       141
   macro avg       0.56      0.55      0.54       141
weighted avg       0.61      0.64      0.61       141

